This program is used to generate a lot a matrice, and stored them as dat file with their 10% best result (indexes)

In [1]:
import qubogen
import numpy as np
import math
import os

from scipy.optimize import minimize
from scipy.spatial.distance import pdist, squareform

In [2]:
from pulser import Pulse, Sequence, Register
from pulser.devices import DigitalAnalogDevice

def evaluate_mapping(new_coords, *args):
    """Cost function to minimize. Ideally, the pairwise
    distances are conserved"""
    Q, shape = args
    new_coords = np.reshape(new_coords, shape)
    new_Q = squareform(
        DigitalAnalogDevice.interaction_coeff / pdist(new_coords) ** 6
    )
    return np.linalg.norm(new_Q - Q)

In [3]:
Q_dict = {}
output_dir = 'matrice'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i in range(4, 13, 2):
    for a in range(1, 11):
        np.random.seed(5)
        #s = np.random.randint(1, 5, i, dtype=int)
        s = np.random.uniform(1, 2, i)
        #s.dtype = 'float32'
        Q = qubogen.qubo_number_partition(s)
        Q = np.array(Q)

        #Check if the QUBO is valid
        try :
            shape = (len(Q), 2)
            costs = []
            np.random.seed()
            x0 = np.random.random(shape).flatten()
            res = minimize(
                evaluate_mapping,
                x0,
                args=(Q, shape),
                method="Nelder-Mead",
                tol=1e-6,
                options={"maxiter": 200000, "maxfev": None},
            )
            coords = np.reshape(res.x, (len(Q), 2))
            qubits = dict(enumerate(coords))
            reg = Register(qubits)
            seq = Sequence(reg, DigitalAnalogDevice)
        except Exception as e:
            print(f"Q{i}_{a} is invalid")
            continue

        key = f"Q{i}_{a}"
        Q_dict[key] = Q

        filename = os.path.join(output_dir, f"{key}.dat")
        np.savetxt(filename, Q, delimiter=',')

for key, value in Q_dict.items():
    array_str = np.array2string(value, separator=', ')
    print(f"{key} = np.array({array_str})")

Q8_4 is invalid
Q8_7 is invalid
Q10_1 is invalid
Q10_8 is invalid
Q12_2 is invalid
Q12_4 is invalid
Q12_5 is invalid
Q12_6 is invalid
Q12_7 is invalid
Q12_9 is invalid
Q12_10 is invalid
Q4_1 = np.array([[-6.1051541 ,  2.2860221 ,  1.47460257,  2.34452943],
 [ 2.2860221 , -8.13267802,  2.25744851,  3.58920741],
 [ 1.47460257,  2.25744851, -6.04727561,  2.31522453],
 [ 2.34452943,  3.58920741,  2.31522453, -8.24896137]])
Q4_2 = np.array([[-6.1051541 ,  2.2860221 ,  1.47460257,  2.34452943],
 [ 2.2860221 , -8.13267802,  2.25744851,  3.58920741],
 [ 1.47460257,  2.25744851, -6.04727561,  2.31522453],
 [ 2.34452943,  3.58920741,  2.31522453, -8.24896137]])
Q4_3 = np.array([[-6.1051541 ,  2.2860221 ,  1.47460257,  2.34452943],
 [ 2.2860221 , -8.13267802,  2.25744851,  3.58920741],
 [ 1.47460257,  2.25744851, -6.04727561,  2.31522453],
 [ 2.34452943,  3.58920741,  2.31522453, -8.24896137]])
Q4_4 = np.array([[-6.1051541 ,  2.2860221 ,  1.47460257,  2.34452943],
 [ 2.2860221 , -8.13267802,  2.2

In [24]:
s.dtype

dtype('float64')

In [25]:
len(Q_dict)

41

In [4]:
def calculate_indexes(Q):
    bitstrings = [np.binary_repr(i, len(Q)) for i in range(2 ** len(Q))]
    costs = []
    
    for b in bitstrings:
        z = np.array(list(b), dtype=int)
        cost = z.T @ Q @ z
        costs.append(cost)
    zipped = zip(bitstrings, costs)
    sort_zipped = sorted(zipped, key=lambda x: x[1])
    indexes = sort_zipped[:10000]
    return indexes
    
results = {}

for key, Q in Q_dict.items():
    indexes = calculate_indexes(Q)
    indexes_key = f"{key}_indexes"
    # variable dynamique exec
    exec(f"{indexes_key} = indexes")
    results[indexes_key] = indexes

for key in results.keys():
    indexes = eval(key)
    print(f"Matrix {key} top 10000 indexes:\n{indexes[:10000]}\n")


Matrix Q4_1_indexes top 10000 indexes:
[('0011', -9.665787911566703), ('1100', -9.665787911566703), ('0110', -9.665056613049309), ('1001', -9.665056613049309), ('1010', -9.20322457319254), ('0101', -9.203224573192538), ('1110', -8.248961370194065), ('0001', -8.248961370194062), ('1011', -8.132678019926034), ('0100', -8.132678019926033), ('1000', -6.105154097812098), ('0111', -6.105154097812097), ('1101', -6.047275609876359), ('0010', -6.047275609876357), ('1111', -3.1086244689504383e-15), ('0000', 0.0)]

Matrix Q4_2_indexes top 10000 indexes:
[('0011', -9.665787911566703), ('1100', -9.665787911566703), ('0110', -9.665056613049309), ('1001', -9.665056613049309), ('1010', -9.20322457319254), ('0101', -9.203224573192538), ('1110', -8.248961370194065), ('0001', -8.248961370194062), ('1011', -8.132678019926034), ('0100', -8.132678019926033), ('1000', -6.105154097812098), ('0111', -6.105154097812097), ('1101', -6.047275609876359), ('0010', -6.047275609876357), ('1111', -3.1086244689504383e-1

In [19]:
Q6_4_indexes

[('001101', -16.82648807924668),
 ('110010', -16.82648807924668),
 ('100101', -16.825657439897896),
 ('011010', -16.82565743989789),
 ('011001', -16.825501369192427),
 ('100110', -16.825501369192427),
 ('110001', -16.823331480215902),
 ('001110', -16.8233314802159),
 ('101001', -16.82295989557609),
 ('010110', -16.822959895576087),
 ('010101', -16.811990931785353),
 ('101010', -16.811990931785353),
 ('000011', -16.62215899065913),
 ('111100', -16.62215899065913),
 ('010011', -16.432307301001075),
 ('101100', -16.432307301001075),
 ('000111', -16.39742553344459),
 ('111000', -16.39742553344459),
 ('011100', -16.309307682590067),
 ('100011', -16.309307682590067),
 ('001011', -16.27327815904361),
 ('110100', -16.27327815904361),
 ('001010', -15.722480378131305),
 ('110101', -15.722480378131305),
 ('100010', -15.670119639748922),
 ('011101', -15.67011963974892),
 ('001001', -15.590754013441634),
 ('110110', -15.59075401344163),
 ('100001', -15.535392746733935),
 ('011110', -15.535392746733

In [7]:
# threshold = -84
# data = Q5_4_indexes

# # Filtrer les éléments selon le seuil
# data = [item for item in data if item[1] <= threshold]

# Q5_4_indexes = data

# print(data)

In [8]:
# first_value = data[0][1]

# type(first_value)

In [9]:
# # Calculer le seuil 10% en dessous
# pourcent = first_value * 0.1
# print(pourcent)
# pourcent = pourcent.round()
# pourcent

In [10]:
# threshold = first_value - pourcent

In [11]:
# threshold

In [12]:
# # Filtrer les éléments selon le seuil
# filtered_data = [item for item in data if item[1] <= threshold]

# print(filtered_data)

In [5]:
def filter_indexes(data, threshold):
    first_value = data[0][1]
    pourcent = first_value * 0.1
    pourcent = pourcent.round()
    threshold = first_value - pourcent
    filtered_data = [item for item in data if item[1] <= threshold]
    return filtered_data

In [14]:
threshold = -70
filter_indexes(Q6_4_indexes, threshold)

[('001110', -118.39100220195982),
 ('110001', -118.39100220195982),
 ('011100', -118.34367713972954),
 ('100011', -118.34367713972952),
 ('010110', -118.29330120150527),
 ('101001', -118.29330120150526),
 ('010101', -117.95218659915878),
 ('101010', -117.95218659915876),
 ('000111', -117.65416873539033),
 ('111000', -117.65416873539033),
 ('110010', -117.48263073043869),
 ('001101', -117.48263073043867),
 ('110100', -117.46099551840888),
 ('001011', -117.46099551840885),
 ('011001', -117.04474626395594),
 ('100110', -117.04474626395594),
 ('101100', -116.81606597346801),
 ('010011', -116.81606597346799),
 ('011011', -115.1949104293557),
 ('100100', -115.1949104293557),
 ('000101', -114.02458996953803),
 ('111010', -114.02458996953803),
 ('011010', -113.82585909615449),
 ('100101', -113.82585909615449),
 ('011110', -110.94040731881736),
 ('100001', -110.94040731881735),
 ('001100', -109.56295940852621),
 ('110011', -109.56295940852621),
 ('111001', -108.9914439934983),
 ('000110', -108.

In [6]:
def filter_indexes(data):
    first_value = data[0][1]
    pourcent = first_value * 0.1
    pourcent = round(pourcent)
    threshold = first_value - pourcent
    filtered_data = [item for item in data if item[1] <= threshold]
    return filtered_data

# Noms des variables générées
variable_names = [f'Q{i}_{j}_indexes' for i in range(4, 13) for j in range(1, 11)]

# Dictionnaire pour stocker les résultats filtrés
filtered_results = {}

# Parcourir les variables globales
for var_name in variable_names:
    if var_name in globals():
        data = globals()[var_name]
        filtered_results[var_name] = filter_indexes(data)

# Créer le dossier "minimal" s'il n'existe pas
output_dir = 'minimal'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Enregistrer les résultats filtrés dans des fichiers .dat
for var_name, filtered_data in filtered_results.items():
    filename = os.path.join(output_dir, f"{var_name}.dat")
    np.savetxt(filename, filtered_data, delimiter=',', fmt='%s')

    # Optionnel : afficher les résultats filtrés
    print(f'{var_name}: {filtered_data}')

Q4_1_indexes: [('0011', -9.665787911566703), ('1100', -9.665787911566703), ('0110', -9.665056613049309), ('1001', -9.665056613049309), ('1010', -9.20322457319254), ('0101', -9.203224573192538)]
Q4_2_indexes: [('0011', -9.665787911566703), ('1100', -9.665787911566703), ('0110', -9.665056613049309), ('1001', -9.665056613049309), ('1010', -9.20322457319254), ('0101', -9.203224573192538)]
Q4_3_indexes: [('0011', -9.665787911566703), ('1100', -9.665787911566703), ('0110', -9.665056613049309), ('1001', -9.665056613049309), ('1010', -9.20322457319254), ('0101', -9.203224573192538)]
Q4_4_indexes: [('0011', -9.665787911566703), ('1100', -9.665787911566703), ('0110', -9.665056613049309), ('1001', -9.665056613049309), ('1010', -9.20322457319254), ('0101', -9.203224573192538)]
Q4_5_indexes: [('0011', -9.665787911566703), ('1100', -9.665787911566703), ('0110', -9.665056613049309), ('1001', -9.665056613049309), ('1010', -9.20322457319254), ('0101', -9.203224573192538)]
Q4_6_indexes: [('0011', -9.665